Import modules

In [1]:
import os
import pandas as pd 
from pathlib import Path
import string
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords
import io

find_files is given a directory. It creates a database containing the name and relative path for each file

In [2]:
 def find_files(subfolder_path):
    #This code creates a database with every file and its path
    dir = Path(os.getcwd())
    #Add directory where your files are:
    newdir = dir / subfolder_path


    #subfolders = os.listdir(newdir)
    subfolders = ['Train_data', 'Test_data']
    dense_list = [os.listdir(newdir / subfolder) for subfolder in subfolders]
    paired_list = zip(dense_list, subfolders)

    audio_files = [(item, label) for sublist, label in paired_list for item in sublist]
    audio_file_list, path_list = zip(*audio_files)
    columns = ['file', 'relative_path']

    common_prefix = os.path.commonprefix([dir, newdir])
    relative_path = os.path.relpath(newdir, common_prefix)
    print(relative_path)

    #Using os.join.path and Path() leads to a Windows path, so I had to do it this way
    relative_path = [relative_path + '/' + path for path in path_list]
    print(relative_path)


    df = pd.DataFrame(columns = columns)
    print(audio_file_list)
    print(relative_path)
    df.file = audio_file_list
    df.relative_path = relative_path
    return df

# df.to_csv(dir / 'audio_database.csv', encoding = 'utf-8')

In [3]:
df = find_files('Feature_representations')

Feature_representations
['Feature_representations/Train_data', 'Feature_representations/Train_data', 'Feature_representations/Test_data', 'Feature_representations/Test_data', 'Feature_representations/Test_data', 'Feature_representations/Test_data', 'Feature_representations/Test_data', 'Feature_representations/Test_data', 'Feature_representations/Test_data', 'Feature_representations/Test_data', 'Feature_representations/Test_data', 'Feature_representations/Test_data', 'Feature_representations/Test_data', 'Feature_representations/Test_data', 'Feature_representations/Test_data', 'Feature_representations/Test_data', 'Feature_representations/Test_data']
('.ipynb_checkpoints', 'Train_1.csv', 'Test_1.csv', 'Test_10.csv', 'Test_11.csv', 'Test_12.csv', 'Test_13.csv', 'Test_14.csv', 'Test_15.csv', 'Test_2.csv', 'Test_3.csv', 'Test_4.csv', 'Test_5.csv', 'Test_6.csv', 'Test_7.csv', 'Test_8.csv', 'Test_9.csv')
['Feature_representations/Train_data', 'Feature_representations/Train_data', 'Feature_repr

In [4]:
# 'test' in df['file'] or 'train' in df['file']
non_ind = list(map(lambda x: 'test' in x or 'train' in x, list(df['file'].apply(lambda x: x.lower()))))
df = df.drop([i for i, x in enumerate(non_ind) if not x])
df.reset_index(drop = True, inplace = True)

In [5]:
test_ind = list(map(lambda x: 'test' in x, list(df['file'].apply(lambda x: x.lower()))))
df['test'] = df['file'][[i for i, x in enumerate(test_ind) if x]]
df['train'] = df['file'][[i for i, x in enumerate(test_ind) if not x]]
# df.drop(['file'], axis = 1, inplace = True)
print(df)

           file                       relative_path         test        train
0   Train_1.csv  Feature_representations/Train_data          NaN  Train_1.csv
1    Test_1.csv   Feature_representations/Test_data   Test_1.csv          NaN
2   Test_10.csv   Feature_representations/Test_data  Test_10.csv          NaN
3   Test_11.csv   Feature_representations/Test_data  Test_11.csv          NaN
4   Test_12.csv   Feature_representations/Test_data  Test_12.csv          NaN
5   Test_13.csv   Feature_representations/Test_data  Test_13.csv          NaN
6   Test_14.csv   Feature_representations/Test_data  Test_14.csv          NaN
7   Test_15.csv   Feature_representations/Test_data  Test_15.csv          NaN
8    Test_2.csv   Feature_representations/Test_data   Test_2.csv          NaN
9    Test_3.csv   Feature_representations/Test_data   Test_3.csv          NaN
10   Test_4.csv   Feature_representations/Test_data   Test_4.csv          NaN
11   Test_5.csv   Feature_representations/Test_data   Test_5.csv

Create_train_test_w2v_matrices takes the dataframe containing the path to each train and test csv file, which is output by find_files. This function completes the processing required to create the word averaged representation of each piece of input data. The train and test sections coerce the input dataframe into the correct format, and the input is then feed to the docAveraging function. The final matrix representation is saved to the disk.

In [51]:
def create_path(*args):
    cur_path = os.getcwd()
    for value in args:
        cur_path  = os.path.join(cur_path, value)
    return cur_path

def lexical_diversity(text):
    full_text = ' '.join(text)
    tokens = nltk.word_tokenize(full_text)
    return len(set(tokens)) / len(tokens)

class Data:
    
    table = str.maketrans({key: None for key in string.punctuation})
    def __init__(self, file, rel_path):
        self.quest_num = int(re.search(r'\d+', file).group())
        if 'test' in file.lower():
            self.mat_type = 'test'
            #Test Set
            test = pd.read_csv(create_path(rel_path, file))
            column_names = test.iloc[0,:]
            test.drop([0], inplace=True)
            test.rename(columns = column_names, inplace=True)
            label_col_bound = len([type(x) for x in test.columns.values if isinstance(x, str)])
            test = test.iloc[:, :label_col_bound]
            test['labels_1'] = test[['Label-1']].apply(lambda x: ''.join(x.astype(str)),axis=1)
            test['labels_2'] = test[['Label-2']].apply(lambda x: ''.join(x.astype(str)),axis=1)
            test = test[~test['labels_1'].str.contains("e")]
            test = test[~test['labels_2'].str.contains("e")]
            self.X = test['Dialogue'].apply(lambda x : x.lower().translate(Data.table))

            test['length'] = self.X.apply(lambda x: len(nltk.word_tokenize(x)))
            test['uniq_length'] = self.X.apply(lambda x: len(set(nltk.word_tokenize(x))))
#             context_len = test[test['Type'] == 'context']['length']
            feedback_len = test[test['Type'] == 'feedback']['length']
            rewrite_len = test[test['Type'] == 'rewrite']['length']
            full_length = pd.concat([feedback_len, rewrite_len])
            self.total_len = len(feedback_len) + len(rewrite_len)
#             self.context_len = len(context_len)
            self.feedback_len = len(feedback_len)
            self.rewrite_len = len(rewrite_len)
#             context_dial = test[test['Type'] == 'context']['Dialogue']
            feedback_dial = test[test['Type'] == 'feedback']['Dialogue']
            rewrite_dial = test[test['Type'] == 'rewrite']['Dialogue']
            full_dial = pd.concat([feedback_dial, rewrite_dial])
#             self.lex_div_context = lexical_diversity(list(context_dial))
            self.lex_div_feedback = lexical_diversity(list(feedback_dial))
            self.lex_div_rewrite = lexical_diversity(list(rewrite_dial))
#             self.lex_div_context_w = self.lex_div_context*(self.context_len/self.total_len)
            self.lex_div_feedback_w = self.lex_div_feedback*(self.feedback_len/self.total_len)
            self.lex_div_rewrite_w = self.lex_div_rewrite*(self.rewrite_len/self.total_len)
            self.lex_div_full = lexical_diversity(list(full_dial))
#             self.context_len_avg = np.average(context_len)
            self.feedback_len_avg = np.average(feedback_len)
            self.rewrite_len_avg = np.average(rewrite_len)
            test['Dialogue'] = test['Dialogue'].apply(lambda x: x.lower().translate(Data.table))
            self.full_len_avg = np.average(full_length)
#             self.Y = test['labels']
            self.full = test


In [36]:
# def create_path(*args):
#     cur_path = os.getcwd()
#     for value in args:
#         cur_path  = os.path.join(cur_path, value)
#     return cur_path

# def lexical_diversity(text):
#     full_text = ' '.join(text)
#     tokens = nltk.word_tokenize(full_text)
#     return len(set(tokens)) / len(tokens)


# class Data:
    
#     table = str.maketrans({key: None for key in string.punctuation})
#     def __init__(self, file, rel_path):
#         self.quest_num = file[-5]
#         if 'test' in file.lower():
#             self.mat_type = 'test'
#             #Test Set
#             test = pd.read_csv(create_path(rel_path, file))
#             column_names = test.iloc[0,:]
#             test.drop([0], inplace=True)
#             test.rename(columns = column_names, inplace=True)
#             test.dropna(inplace=True)
# #             test['labels'] = test[test.columns[2:]].apply(
# #                 lambda x: ''.join(x.astype(str)),axis=1)
# #             self.num_labels = len(max(list(test['labels']), key=len))
# #             test.drop(test[test.labels=='e'*self.num_labels].index, inplace=True)
#             self.X = test['Dialogue'].apply(lambda x : x.lower().translate(Data.table))
# #             test['stop_word_count'] = self.X.apply(lambda x: len([w for w in nltk.word_tokenize(x) if w in stop_words]))
#             #Splitting the text by type: 
#             context_dial = test[test['Type'] == 'context']['Dialogue']
#             feedback_dial = test[test['Type'] == 'feedback']['Dialogue']
#             rewrite_dial = test[test['Type'] == 'rewrite']['Dialogue']
#             self.lex_div_context = lexical_diversity(list(context_dial))
#             self.lex_div_feedback = lexical_diversity(list(feedback_dial))
#             self.lex_div_rewrite = lexical_diversity(list(rewrite_dial))
#             self.lex_div_full = lexical_diversity(self.X)
#             test['length'] = self.X.apply(lambda x: len(nltk.word_tokenize(x)))
#             test['uniq_length'] = self.X.apply(lambda x: len(set(nltk.word_tokenize(x))))
#             context_len = test[test['Type'] == 'context']['length']
#             feedback_len = test[test['Type'] == 'feedback']['length']
#             rewrite_len = test[test['Type'] == 'rewrite']['length']
#             self.context_len = np.average(context_len)
#             self.feedback_len = np.average(feedback_len)
#             self.rewrite_len = np.average(rewrite_len)
#             test['Dialogue'] = test['Dialogue'].apply(lambda x: x.lower().translate(Data.table))
#             self.full_len = np.average(test['length'])
#             self.Y = test['labels']
#             self.full = test
                        

#         #Train set
#         elif 'train' in file.lower():
#             self.mat_type = 'train'
#             #Train set
#             train = pd.read_csv(create_path(rel_path, file))
#             train.drop([train.columns[0]], axis = 1, inplace=True)
#             train.dropna(inplace=True)
#             train['labels']=train.labels.apply(lambda x: ''.join([(3-len(str(x)))*'0',str(x)]))
#             self.X = train['Dialogue'].apply(lambda x : x.lower().translate(Data.table))
#             self.Y = train['labels']
            
# def save_labels(Y, out_path):
#     if not os.path.isfile(out_path):
#         np.save(out_path, Y)

This cell calls the class Data

In [85]:
test = pd.read_csv(create_path(df.relative_path[i], df.file[i]))
column_names = test.iloc[0,:]
test.drop([0], inplace=True)
test.rename(columns = column_names, inplace=True)
label_col_bound = len([type(x) for x in test.columns.values if isinstance(x, str)])
test = test.iloc[:, :label_col_bound]
test['labels_1'] = test[['Label-1']].apply(lambda x: ''.join(x.astype(str)),axis=1)
test['labels_2'] = test[['Label-2']].apply(lambda x: ''.join(x.astype(str)),axis=1)
test = test[~test['labels_1'].str.contains("e")]
test = test[~test['labels_2'].str.contains("e")]
test

,Assignment,Type,Dialogue,Label-1,Label-2,Label-1,Label-2,Label-1,Label-2,labels_1,labels_2
1,3137ONMDKG55SOUPVZ46INTDKW3GE9a,rewrite,I hope everything was figured out. I would lik...,1,1,0,0,0,0,100,100
2,3137ONMDKG55SOUPVZ46INTDKW3GE9b,rewrite,I wanted to go over some ideas I had for what ...,1,1,1,1,0,0,110,110
3,3137ONMDKG55SOUPVZ46INTDKW3GE9c,rewrite,"Oh, is it about your flight delay? We have eno...",1,1,1,1,1,1,111,111
4,3137ONMDKG55SOUPVZ46INTDKW3GE9d,rewrite,"I am sorry for my interjection, but this sound...",1,1,1,1,0,1,110,111
5,3137ONMDKG55SOUPVZ46INTDKWRGEXa,rewrite,I hope everything worked itself out. If now's ...,1,1,0,0,0,0,100,100
6,3137ONMDKG55SOUPVZ46INTDKWRGEXb,rewrite,"I heard your plane was delayed, so I thought I...",1,1,1,1,0,0,110,110
7,3137ONMDKG55SOUPVZ46INTDKWRGEXc,rewrite,Everything will work out. There's plenty of su...,0,0,1,1,1,1,011,011
8,3137ONMDKG55SOUPVZ46INTDKWRGEXd,rewrite,Sorry to interrupt something that seemed impor...,1,1,1,1,0,0,110,110
9,31UV0MXWNQCJSW7RAMB6Z50GMGK5IDa,rewrite,I just received word that you had a significan...,1,1,0,0,0,0,100,100
10,31UV0MXWNQCJSW7RAMB6Z50GMGK5IDb,rewrite,This plane delay is a good opportunity to work...,1,1,1,1,0,0,110,110


In [52]:
total_df = pd.DataFrame(columns = ['Quest_num', 'Type', 'Lex_Div', 'Lex_Div W', 'Full Length', 'Avg_Len'])
for i in range(1, len(df)):
    print(df.file[i])
    print(i)
    cur_data = Data(df.file[i], df.relative_path[i])
    new_df = pd.DataFrame({'Quest_num': [cur_data.quest_num]*3,
                  'Type' : ['feedback', 'rewrite', 'full'], 
                  'Lex_Div' : [cur_data.lex_div_feedback, cur_data.lex_div_rewrite, cur_data.lex_div_full],
                  'Lex_Div W' : [cur_data.lex_div_feedback_w, cur_data.lex_div_rewrite_w, cur_data.lex_div_full],
                  'Full Length' : [cur_data.feedback_len, cur_data.rewrite_len, cur_data.total_len],
                  'Avg_Len' : [cur_data.feedback_len_avg, cur_data.rewrite_len_avg, cur_data.full_len_avg]})
    total_df = pd.concat([total_df, new_df])
total_df
total_df.to_csv('lexical_diversity_per_question.csv')

Test_1.csv
1
Test_10.csv
2
Test_11.csv
3
Test_12.csv
4
Test_13.csv
5
Test_14.csv
6
Test_15.csv
7
Test_2.csv
8
Test_3.csv
9
Test_4.csv
10
Test_5.csv
11
Test_6.csv
12
Test_7.csv
13
Test_8.csv
14
Test_9.csv
15


In [53]:
total_df

,Quest_num,Type,Lex_Div,Lex_Div W,Full Length,Avg_Len
0,1,feedback,0.139891,0.063641,106,6.660377
1,1,rewrite,0.092508,0.050423,127,8.283465
2,1,full,0.079622,0.079622,233,7.545064
0,10,feedback,0.113151,0.069631,192,13.562500
1,10,rewrite,0.132880,0.051108,120,11.850000
2,10,full,0.085826,0.085826,312,12.903846
0,11,feedback,0.164661,0.099019,89,15.303371
1,11,rewrite,0.194030,0.077350,59,9.305085
2,11,full,0.130725,0.130725,148,12.912162
0,12,feedback,0.136442,0.081616,131,19.282443


In [ ]:
file = create_path('clean_data','question_{}_clean_data.csv'.format(cur_data.quest_num))
cur_data.full.to_csv(file)

In [118]:
cur_data.quest_num

13

In [245]:
for i in range(1, len(df)):
    if '8' not in df.file[i]:
        cur_data = Data(df.file[i], df.relative_path[i])
        index = 0
        stop_words = set(stopwords.words('english')) 
        df_out = pd.DataFrame(columns = ['Words', 'Index'])
        for entry in cur_data.full['Dialogue']:
            sent = nltk.word_tokenize(entry)
            for word in sent:
                if word not in stop_words:
                    d = {'Words': [word], 'Index': [index]}
                    df_out = df_out.append(pd.DataFrame(data = d))
            index += 1
        df_out

        df_out.Words.reset_index(inplace = True, drop = True)
        # pd.DataFrame(cur_data.full.iloc[0,2:]).transpose()
        # pd.DataFrame(df_out.iloc[0,:]).transpose()['Words']
        # # pd.DataFrame(cur_data.full.iloc[0,2:]).transpose()
        # # pd.DataFrame(df_out.iloc[0,:]).transpose().join(pd.DataFrame(cur_data.full.iloc[0,2:]).transpose())
        # pd.DataFrame(pd.concat([pd.Series(df_out.iloc[0,:]), pd.Series(cur_data.full.iloc[0,2:])], axis = 0)).transpose()
        path = create_path('new_dialogue_features.csv')
        features = pd.read_csv(path)
        features.columns.values
        features.columns = ['Section', 'Feature_1', 'Feature_2', 'Feature_3', 'Feature_4']
        features.reset_index(inplace=True, drop = True)

        final_df = pd.DataFrame()
        for i in range(len(df_out)):
            ind = int(df_out.iloc[i,1])
            new_line = pd.DataFrame(pd.concat([pd.Series(df_out.iloc[i,:]), pd.Series(cur_data.full.iloc[ind,2:])], axis = 0)).transpose()
            final_df = final_df.append(new_line)
        final_df

        for i in range(1, len(features.iloc[cur_data.quest_num-1,:].dropna())):
            names = final_df.columns.tolist()
            names[names.index('Label-1')] = features.iloc[cur_data.quest_num-1,i] + '_1'
            names[names.index('Label-2')] = features.iloc[cur_data.quest_num-1, i] + '_2'
            final_df.columns = names

        final_df

        final_df.to_csv('word_cloud_question_{}.csv'.format(cur_data.quest_num))

In [241]:
len(features.iloc[cur_data.quest_num,:].dropna())

3

In [242]:
features.iloc[cur_data.quest_num,:]

Section                                                     12
Feature_1    Greets or appreciates the officer for meeting ...
Feature_2    Asks for the reason why he chose his plan (e.g...
Feature_3                                                  NaN
Feature_4                                                  NaN
Name: 11, dtype: object

In [229]:
final_df

,Words,Index,Dialogue,Greets or appreciates the officer for meeting with you (e.g. “Thanks for meeting with me”),Greets or appreciates the officer for meeting with you (e.g. “Thanks for meeting with me”),labels_1,labels_2,length,uniq_length
0,wont,0,this wont take too long,1,1,1,1,5,5
0,take,0,this wont take too long,1,1,1,1,5,5
0,long,0,this wont take too long,1,1,1,1,5,5
0,dont,1,we dont have much time does this have to be in...,0,0,0,0,12,11
0,much,1,we dont have much time does this have to be in...,0,0,0,0,12,11
0,time,1,we dont have much time does this have to be in...,0,0,0,0,12,11
0,private,1,we dont have much time does this have to be in...,0,0,0,0,12,11
0,would,2,it would not be long at all,1,1,1,1,7,7
0,long,2,it would not be long at all,1,1,1,1,7,7
0,need,3,this does not need to be in private we do not ...,0,0,0,0,15,13
